In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
from IPython.display import Javascript
from IPython.display import Image as Image2
from IPython import display
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
from PIL import Image as Image1
import gc

In [ ]:
#Устанавливаем Yolo V5 (large) для распознавания

detector = torch.hub.load('ultralytics/yolov5', 'custom',
                              path = 'yolov5m.pt') #'yolov5s.pt'
print('#'*60)
print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'}), OpenCV: {cv2.__version__}")

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 308.3 MB/s eta

############################################################
Using torch 2.0.1+cu118 (CPU), OpenCV: 4.7.0


In [ ]:
# Список детектируемых классов
coco_labels = '''person
bicycle
car
motorcycle
airplane
bus
train
truck
boat
traffic light
fire hydrant
street sign
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
hat
backpack
umbrella
shoe
eye glasses
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
plate
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
couch
potted plant
bed
mirror
dining table
window
desk
toilet
door
tv
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
blender
book
clock
vase
scissors
teddy bear
hair drier
toothbrush
hair brush'''.split()

In [ ]:
# функция преобразования объекта JavaScript в изображение OpenCV image - надо запускать также для следующего раздела
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # декодируем изображение из формата base64
  image_bytes = b64decode(js_reply.split(',')[1])
  # конвертируем байты в numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # декодируем numpy array в изображение OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# функция для преобразования изображения с прямоугольником OpenCV Rectangle bounding box image в байтовую строку base64 чтобы вывести в видеопоток
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # конвертируем массив в изображение PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # преобразуем bbox в формат png для возврата из функции с помощью return
  bbox_PIL.save(iobuf, format='png')
  # форматируем возвращаемую с помощью return строку
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# Используем код на JavaScript для захвата потока с нашей вебкамеры и передачи данных на облачный компьютер - надо запускать также для следующего раздела
def video_stream_js():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'Чтобы завершить, щёлкните мышкой на данном видео</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display.display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

Детекция

In [ ]:
# получаем прямоугольники(баундбоксы) - в кадре должен быть человек
# очищаем память
gc.collect()

# начинаем захват видео с камеры
video_stream_js()

# Создаём надпись
label_html = 'Захват изображения...'

# инициализируем пустые прямоугольники выделения
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # конвертируем поток, передаваемый через JS, в изображение OpenCV Image
    img = js_to_image(js_reply["img"])

    # создаём пустой массив, в котором будет отображаться детекция объектов
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # переводим изображение в серое для детекции
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # детектирование
    cur = detector(img) #img
    yolo_preds = cur.xyxy[0].cpu().numpy()

    # получение коодинат прямоугольников
    boxes = yolo_preds[:, :4]  # xmin, ymin, xmax, ymax
    confs = yolo_preds[:, 4]
    labels = yolo_preds[:, 5]

    fst_box_flag = 1

    # получаем конкретные прямоугольники
    for box, conf, label in zip(boxes, confs, labels):
        if coco_labels[int(label)] == 'person' and fst_box_flag == 1: #
             y1, y2, x1, x2 = int(box[1]), int(box[3]), int(box[0]), int(box[2])
             bbox_array = cv2.rectangle(bbox_array,(x1,y1),(x2,y2),(0,255,0),2)
             fst_box_flag = 0

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255

    bbox_bytes = bbox_to_bytes(bbox_array) #bbox_array
    # записываем в конкретные массивы изображения с прямоугольниками
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

https://drive.google.com/file/d/1mkjCl3jJS6CLBpjtT8lD4JrLdFeubEX2/view?usp=sharing

Сегментация

In [ ]:
!git clone -b shapemask https://github.com/tensorflow/tpu/

Cloning into 'tpu'...
remote: Enumerating objects: 11310, done.
remote: Counting objects: 100% (1512/1512), done.
remote: Compressing objects: 100% (754/754), done.
remote: Total 11310 (delta 877), reused 1271 (delta 720), pack-reused 9798
Receiving objects: 100% (11310/11310), 47.03 MiB | 34.57 MiB/s, done.
Resolving deltas: 100% (7915/7915), done.


In [ ]:
# смещаем указатель в удобную папку
%cd '/content/tpu/models/official/detection/utils/object_detection'

/content/tpu/models/official/detection/utils/object_detection


In [ ]:
# переделываем часть строк в файлах Python чтобы не получать ошибку из-за дублирования названия функции из ultralytics
with open ('/content/tpu/models/official/detection/utils/object_detection/visualization_utils.py', 'r') as f:
  old_data = f.read()

new_data = old_data.replace('from utils.object_detection import shape_utils', 'import shape_utils')

with open ('/content/tpu/models/official/detection/utils/object_detection/visualization_utils.py', 'w') as f:
  f.write(new_data)


with open ('/content/tpu/models/official/detection/evaluation/coco_utils.py', 'r') as f:
  old_data = f.read()

new_data = old_data.replace('from utils import box_utils', 'from detection.utils1 import box_utils')

with open ('/content/tpu/models/official/detection/evaluation/coco_utils.py', 'w') as f:
  f.write(new_data)

In [ ]:
# переносим папку с функциями чтобы они не пересекались по названию с ultralytics
!mv '/content/tpu/models/official/detection/utils/' '/content/tpu/models/official/detection/utils1/'

In [ ]:
import tensorflow.compat.v1 as tf
from PIL import Image as Image1
import sys

# добавляем системные пути чтобы найти модули
sys.path.insert(0, '/content/tpu/models/official')
sys.path.insert(0, '/content/tpu/models/official/detection')
sys.path.insert(0, '/content/tpu/models/official/detection/utils')
sys.path.insert(0, '/content/tpu/models/official/detection/utils/object_detection')
sys.path.insert(0, '/content/tpu/models/official/detection/utils1')
sys.path.insert(0, '/content/tpu/models/official/detection/utils1/object_detection')

# теперь импортируем перемещённые модули
from utils1.object_detection import visualization_utils
from evaluation import coco_utils

In [ ]:
ID_MAPPING = {
    1: 'person',
    2: 'bicycle',
    3: 'car',
    4: 'motorcycle',
    5: 'airplane',
    6: 'bus',
    7: 'train',
    8: 'truck',
    9: 'boat',
    10: 'traffic light',
    11: 'fire hydrant',
    13: 'stop sign',
    14: 'parking meter',
    15: 'bench',
    16: 'bird',
    17: 'cat',
    18: 'dog',
    19: 'horse',
    20: 'sheep',
    21: 'cow',
    22: 'elephant',
    23: 'bear',
    24: 'zebra',
    25: 'giraffe',
    27: 'backpack',
    28: 'umbrella',
    31: 'handbag',
    32: 'tie',
    33: 'suitcase',
    34: 'frisbee',
    35: 'skis',
    36: 'snowboard',
    37: 'sports ball',
    38: 'kite',
    39: 'baseball bat',
    40: 'baseball glove',
    41: 'skateboard',
    42: 'surfboard',
    43: 'tennis racket',
    44: 'bottle',
    46: 'wine glass',
    47: 'cup',
    48: 'fork',
    49: 'knife',
    50: 'spoon',
    51: 'bowl',
    52: 'banana',
    53: 'apple',
    54: 'sandwich',
    55: 'orange',
    56: 'broccoli',
    57: 'carrot',
    58: 'hot dog',
    59: 'pizza',
    60: 'donut',
    61: 'cake',
    62: 'chair',
    63: 'couch',
    64: 'potted plant',
    65: 'bed',
    67: 'dining table',
    70: 'toilet',
    72: 'tv',
    73: 'laptop',
    74: 'mouse',
    75: 'remote',
    76: 'keyboard',
    77: 'cell phone',
    78: 'microwave',
    79: 'oven',
    80: 'toaster',
    81: 'sink',
    82: 'refrigerator',
    84: 'book',
    85: 'clock',
    86: 'vase',
    87: 'scissors',
    88: 'teddy bear',
    89: 'hair drier',
    90: 'toothbrush',
}
category_index = {k: {'id': k, 'name': ID_MAPPING[k]} for k in ID_MAPPING}

In [ ]:
#use_tpu = True #@param {type:"boolean"}
use_tpu = False # мы делаем на GPU, поэтому отключаем поддержку TPU
if use_tpu:
  import os
  import pprint

  assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)

  session = tf.Session(TPU_ADDRESS, graph=tf.Graph())
  print('TPU devices:')
  pprint.pprint(session.list_devices())
else:
  session = tf.Session(graph=tf.Graph())

In [ ]:
saved_model_dir = 'gs://cloud-tpu-checkpoints/shapemask/1571767330' #@param {type:"string"}
_ = tf.saved_model.loader.load(session, ['serve'], saved_model_dir)

Instructions for updating:
Use `tf.saved_model.load` instead.


In [ ]:
max_boxes_to_draw = 1   #@param {type:"integer"}
# показываем только один прямоугольник

min_score_thresh = 0.23    #@param {type:"slider", min:0, max:1, step:0.01}
#минимальный балл(порог)

# получаем прямоугольники(баундбоксы) и сегментацию - в кадре должен быть человек
# очищаем память
gc.collect()

# начинаем захват видео с камеры
video_stream_js()

# Создаём надпись
label_html = 'Захват видео...'

# инициализируем пустые прямоугольники выделения
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # конвертируем поток, передаваемый через JS, в изображение OpenCV Image
    img = js_to_image(js_reply["img"])

    # создаём пустой массив, в котором будет отображаться детекция объектов
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # переводим изображение в серое для детекции
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    cv2.rectangle(img, (25,25), (115,115), (128,0,128), thickness=2, lineType=8, shift=0)
    cv2.putText(img, "1T", (30,90), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,0,255), 2)

    #cv2_imshow('result', img)

    # детектирование
    height, width, _ = img.shape
    Image1.fromarray(img).save(f"/content/1.jpg")

    with open(f"/content/1.jpg", 'rb') as f:
        np_image_string_idx = np.array([f.read()])

    num_detections, detection_boxes, detection_classes, detection_scores, detection_masks, detection_outer_boxes, image_info = session.run(
        ['NumDetections:0', 'DetectionBoxes:0', 'DetectionClasses:0', 'DetectionScores:0',
         'DetectionMasks:0', 'DetectionOuterBoxes:0', 'ImageInfo:0'],
        feed_dict={'Placeholder:0': np_image_string_idx})

    num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
    detection_boxes = np.squeeze(detection_boxes / min(image_info[0, 2]), axis=(0,))[0:num_detections]
    detection_outer_boxes = np.squeeze(detection_outer_boxes / min(image_info[0, 2]), axis=(0,))[0:num_detections]
    detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
    detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
    instance_masks = np.squeeze(detection_masks, axis=(0,))[0:num_detections]

    # Используем внешние прямоугольники (баундбоксы) и сегментируем
    ymin, xmin, ymax, xmax = np.split(detection_outer_boxes, 4, axis=-1)
    processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
    segmentations = coco_utils.generate_segmentation_from_masks(instance_masks, processed_boxes, height, width)

    bbox_array = visualization_utils.visualize_boxes_and_labels_on_image_array(
        img * 1,
        detection_boxes,
        detection_classes,
        detection_scores,
        category_index,
        instance_masks=segmentations, # инстанс сегментация
        use_normalized_coordinates=False,
        max_boxes_to_draw=max_boxes_to_draw,
        min_score_thresh=min_score_thresh
        )
    bbox_array = np.concatenate((bbox_array, bbox_array[:,:,0].reshape(height, width, 1)), axis = 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255

    bbox_bytes = bbox_to_bytes(bbox_array) #bbox_array

    # записываем в конкретные массивы изображения с прямоугольниками и сегментацией
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

https://drive.google.com/file/d/1jEzYjF70KMxPGw27YXRPsVaehc51vW0D/view?usp=sharing